In [1]:
from greensight.utils import DIR_DATA
import pandas as pd
import numpy as np
from datetime import datetime
import json
from typing import Union
from pathlib import Path
import re
from tqdm.notebook import tqdm

In [2]:
sentinel_data_path = DIR_DATA / "indices"
assert sentinel_data_path.is_dir()

In [3]:
year_paths = [path for path in sorted(list(sentinel_data_path.iterdir())) if "2015" not in path.stem and "2016" not in path.stem]
year_paths

[PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2017_polygon_bands_and_indices'),
 PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2018_polygon_bands_and_indices'),
 PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2019_polygon_bands_and_indices'),
 PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2020_polygon_bands_and_indices'),
 PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2021_polygon_bands_and_indices'),
 PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2022_polygon_bands_and_indices'),
 PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2023_polygon_bands_and_indices'),
 PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2024_polygon_bands_and_indices')]

In [4]:
def load_sentinel_two_index_data_from_csv(path: Union[str, Path]) -> pd.DataFrame:

    """""
    loads sentinel two data from file path and process into dataframe 
    """
    path = Path(path)
    assert path.is_file()

    df = pd.read_csv(path)

    # Extract the year using regex
    match = re.search(r"\d{4}", str(path))
    year = int(match.group(0) if match else None)

    df = df.drop(columns=["system:index", ".geo"])
    # set index
    df = df.set_index("LAD_CD")

    # check for duplicates
    assert np.unique(df.columns).shape == df.columns.shape
    index_inds = list(set(
        sorted([i.split("_")[1] for i in df.columns.unique() if i.split("_")[0].isnumeric()])
    ))
    month_inds = set(
        [i.split("_")[0] for i in df.columns.unique() if i.split("_")[0].isnumeric()]
    )

    months = []
    inds = []
    for month in month_inds:
        # generate desired columns
        required_cols = [month + "_" + band for band in index_inds]

        cols = [col for col in required_cols if col in df.columns] 
        df_month = df[cols].copy()

        # convert from a DataFrame of rows: shapes, columns: bands for a single month to a single row of rows: month, columns: (shape, band)
        row_month = df_month.stack().to_frame().T

                # create multi-index for the columns (shape, band)
        new_cols = [(a, b.split("_")[1]) for a, b in row_month.columns]
        row_month.columns = pd.MultiIndex.from_tuples(new_cols)

        # add to stack
        months.append(row_month)
        # add month name to index.
        inds.append(month)

    # combine rows
    df_month = pd.concat(months, axis=0)

    # fix index to month value
    df_month.index = np.array(inds).astype(int) + 1

    # format index
    df_month = df_month.sort_index()
    df_month.index.name = "date"
    df_month.index = [datetime(year, int(month), 1) for month in df_month.index]
    df_month.columns.names = ("shape", "band")

    assert df_month.shape == (len(month_inds), len(index_inds)*df.shape[0])

    # add greenbelt information from json dict.
    lookup_path = DIR_DATA / "id_lookup/id_lookup.json"
    with open(lookup_path, "r") as in_file:
        D_lookup = json.load(in_file)
    greenbelts = [D_lookup[code]["GB_Name"] for code, _ in df_month.columns]

    # add greenbelts to column MultiIndex
    df_month.columns = pd.MultiIndex.from_tuples(
        [(gb, *cols) for gb, cols in zip(greenbelts, df_month.columns)]
    )
    df_month.columns.names = ("greenbelt", "shape", "band")

    return df_month

In [5]:
df_all = []
for year_path in tqdm(year_paths):
    year_files = sorted([file for file in year_path.iterdir()])

    df_year = []
    for file in tqdm(year_files):
        try: 
            out = load_sentinel_two_index_data_from_csv(file)
            if out is not None:
                df_year.append(out)
        except:
            print(f"Failed file {file}")
    
    if df_year != []:
        df_year = pd.concat(df_year, axis=1)

        df_year = df_year.loc[:, ~df_year.columns.duplicated()]
        df_all.append(df_year)


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

Failed file /home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2017_polygon_bands_and_indices/feature_vectors_S2_0000000185_0000000185_S2_mean.csv


  0%|          | 0/93 [00:00<?, ?it/s]

  0%|          | 0/93 [00:00<?, ?it/s]

...

Failed file /home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2022_polygon_bands_and_indices/feature_vectors_S2_0000000071_0000000073_S2_mean.csv
Failed file /home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2022_polygon_bands_and_indices/feature_vectors_S2_0000000086_0000000088_S2_mean.csv
Failed file /home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2022_polygon_bands_and_indices/feature_vectors_S2_0000000185_0000000185_S2_mean.csv


  0%|          | 0/93 [00:00<?, ?it/s]

Failed file /home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2023_polygon_bands_and_indices/feature_vectors_S2_0000000072_0000000073_S2_mean.csv


  0%|          | 0/93 [00:00<?, ?it/s]

Failed file /home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2024_polygon_bands_and_indices/feature_vectors_S2_0000000072_0000000073_S2_mean.csv


In [ ]:
# # remove duplicate column indices
# for i, df in enumerate(df_all):
#     df_all[i] = df.loc[:, ~df.columns.duplicated()]

In [6]:
df_out = pd.concat(df_all, axis=0)

In [7]:
df_out

greenbelt  Bath and Bristol                                                    \
shape             E06000022                                                     
band                  msavi          pvi         mcari           bi     ndwi2   
2017-04-01        94.486726  3937.384134  65370.597888 -2951.914648 -0.493598   
2017-05-01       121.030270  4135.544158  51998.382445 -3145.115897 -0.482986   
2017-06-01        92.271443  4150.948900  40180.548951 -2950.267034 -0.493938   
2017-07-01        15.527558  3789.197962  38911.867444 -2696.027001 -0.464355   
2017-08-01              NaN          NaN           NaN          NaN       NaN   
...                     ...          ...           ...          ...       ...   
2024-08-01       217.549152  3360.679176  18173.069533 -1971.408288 -0.458801   
2024-09-01       172.578872  3293.176882  18559.690544 -1995.008918 -0.464086   
2024-10-01       131.172695  3128.474663  23134.724415 -2060.726741 -0.439133   
2024-11-01       258.295717  2709.468640  19269.664957 -1753.927588 -0.436032   
2024-12-01        50.240398  2530.506753  15761.605438 -1607.023690 -0.429346   

greenbelt                                                             ...  \
shape                                                                 ...   
band            msavi2     s2rep         wdvi     gndvi          bi2  ...   
2017-04-01   94.486726  0.337793  3957.022082  0.698364 -2951.914648  ...   
2017-05-01  121.030270  0.359582  4156.170441  0.701280 -3145.115897  ...   
2017-06-01   92.271443  0.295201  4171.652016  0.670956 -2950.267034  ...   
2017-07-01   15.527558  0.251190  3808.096822  0.614131 -2696.027001  ...   
2017-08-01         NaN       NaN          NaN       NaN          NaN  ...   
...                ...       ...          ...       ...          ...  ...   
2024-08-01  217.549152  0.211111  3377.440772  0.602442 -1971.408288  ...   
2024-09-01  172.578872  0.213965  3309.601806  0.605956 -1995.008918  ...   
2024-10-01  131.172695  0.259195  3144.078124  0.615078 -2060.726741  ...   
2024-11-01  258.295717  0.240204  2722.982283  0.599835 -1753.927588  ...   
2024-12-01   50.240398  0.239063  2543.127812  0.597630 -1607.023690  ...   

greenbelt       York                                                    \
shape      E06000014                                                     
band            ndvi      ndwi      savi      ipvi        ci      gemi   
2017-04-01  0.419859 -0.434996  0.629708  0.328669  0.419859 -0.831839   
2017-05-01  0.754145 -0.699359  1.131079  0.671939  0.754145 -0.431212   
2017-06-01       NaN       NaN       NaN       NaN       NaN       NaN   
2017-07-01       NaN       NaN       NaN       NaN       NaN       NaN   
2017-08-01  0.621914 -0.604316  0.932758  0.527854  0.621914 -0.369069   
...              ...       ...       ...       ...       ...       ...   
2024-08-01  0.549143 -0.544992  0.823604  0.459687  0.549143 -0.388129   
2024-09-01  0.564789 -0.562622  0.847057  0.480541  0.564789 -0.267417   
2024-10-01  0.588446 -0.560214  0.882515  0.493619  0.588446 -0.361902   
2024-11-01  0.618103 -0.598243  0.926893  0.523736  0.618103 -0.375574   
2024-12-01  0.590813 -0.548785  0.886046  0.490100  0.590813 -0.395506   

greenbelt                            Burton-upon-Trent and Swadlincote         
shape                                                        E07000193         
band              msavi       msavi2                             msavi msavi2  
2017-04-01          NaN          NaN                               NaN    NaN  
2017-05-01   751.027167   751.027167                               NaN    NaN  
2017-06-01          NaN          NaN                               NaN    NaN  
2017-07-01          NaN          NaN                               NaN    NaN  
2017-08-01          NaN          NaN                               NaN    NaN  
...                 ...          ...                               ...    ...  
2024-08-01  1025.252629  1025

In [8]:
# df_out = df_out.interpolate(axis=0)

In [8]:
df_out

greenbelt  Bath and Bristol                                                    \
shape             E06000022                                                     
band                  msavi          pvi         mcari           bi     ndwi2   
2017-04-01        94.486726  3937.384134  65370.597888 -2951.914648 -0.493598   
2017-05-01       121.030270  4135.544158  51998.382445 -3145.115897 -0.482986   
2017-06-01        92.271443  4150.948900  40180.548951 -2950.267034 -0.493938   
2017-07-01        15.527558  3789.197962  38911.867444 -2696.027001 -0.464355   
2017-08-01              NaN          NaN           NaN          NaN       NaN   
...                     ...          ...           ...          ...       ...   
2024-08-01       217.549152  3360.679176  18173.069533 -1971.408288 -0.458801   
2024-09-01       172.578872  3293.176882  18559.690544 -1995.008918 -0.464086   
2024-10-01       131.172695  3128.474663  23134.724415 -2060.726741 -0.439133   
2024-11-01       258.295717  2709.468640  19269.664957 -1753.927588 -0.436032   
2024-12-01        50.240398  2530.506753  15761.605438 -1607.023690 -0.429346   

greenbelt                                                             ...  \
shape                                                                 ...   
band            msavi2     s2rep         wdvi     gndvi          bi2  ...   
2017-04-01   94.486726  0.337793  3957.022082  0.698364 -2951.914648  ...   
2017-05-01  121.030270  0.359582  4156.170441  0.701280 -3145.115897  ...   
2017-06-01   92.271443  0.295201  4171.652016  0.670956 -2950.267034  ...   
2017-07-01   15.527558  0.251190  3808.096822  0.614131 -2696.027001  ...   
2017-08-01         NaN       NaN          NaN       NaN          NaN  ...   
...                ...       ...          ...       ...          ...  ...   
2024-08-01  217.549152  0.211111  3377.440772  0.602442 -1971.408288  ...   
2024-09-01  172.578872  0.213965  3309.601806  0.605956 -1995.008918  ...   
2024-10-01  131.172695  0.259195  3144.078124  0.615078 -2060.726741  ...   
2024-11-01  258.295717  0.240204  2722.982283  0.599835 -1753.927588  ...   
2024-12-01   50.240398  0.239063  2543.127812  0.597630 -1607.023690  ...   

greenbelt       York                                                    \
shape      E06000014                                                     
band            ndvi      ndwi      savi      ipvi        ci      gemi   
2017-04-01  0.419859 -0.434996  0.629708  0.328669  0.419859 -0.831839   
2017-05-01  0.754145 -0.699359  1.131079  0.671939  0.754145 -0.431212   
2017-06-01       NaN       NaN       NaN       NaN       NaN       NaN   
2017-07-01       NaN       NaN       NaN       NaN       NaN       NaN   
2017-08-01  0.621914 -0.604316  0.932758  0.527854  0.621914 -0.369069   
...              ...       ...       ...       ...       ...       ...   
2024-08-01  0.549143 -0.544992  0.823604  0.459687  0.549143 -0.388129   
2024-09-01  0.564789 -0.562622  0.847057  0.480541  0.564789 -0.267417   
2024-10-01  0.588446 -0.560214  0.882515  0.493619  0.588446 -0.361902   
2024-11-01  0.618103 -0.598243  0.926893  0.523736  0.618103 -0.375574   
2024-12-01  0.590813 -0.548785  0.886046  0.490100  0.590813 -0.395506   

greenbelt                            Burton-upon-Trent and Swadlincote         
shape                                                        E07000193         
band              msavi       msavi2                             msavi msavi2  
2017-04-01          NaN          NaN                               NaN    NaN  
2017-05-01   751.027167   751.027167                               NaN    NaN  
2017-06-01          NaN          NaN                               NaN    NaN  
2017-07-01          NaN          NaN                               NaN    NaN  
2017-08-01          NaN          NaN                               NaN    NaN  
...                 ...          ...                               ...    ...  
2024-08-01  1025.252629  1025

In [9]:
# clean up greenbelt place names
df_out.columns = pd.MultiIndex.from_tuples([(i[0].replace("-", " ").replace(",", "") , i[1], i[2]) for i in df_out.columns])
df_out.columns.names = ["greenbelt", "shape", "band"]
df_out.head()

greenbelt  Bath and Bristol                                                    \
shape             E06000022                                                     
band                  msavi          pvi         mcari           bi     ndwi2   
2017-04-01        94.486726  3937.384134  65370.597888 -2951.914648 -0.493598   
2017-05-01       121.030270  4135.544158  51998.382445 -3145.115897 -0.482986   
2017-06-01        92.271443  4150.948900  40180.548951 -2950.267034 -0.493938   
2017-07-01        15.527558  3789.197962  38911.867444 -2696.027001 -0.464355   
2017-08-01              NaN          NaN           NaN          NaN       NaN   

greenbelt                                                             ...  \
shape                                                                 ...   
band            msavi2     s2rep         wdvi     gndvi          bi2  ...   
2017-04-01   94.486726  0.337793  3957.022082  0.698364 -2951.914648  ...   
2017-05-01  121.030270  0.359582  4156.170441  0.701280 -3145.115897  ...   
2017-06-01   92.271443  0.295201  4171.652016  0.670956 -2950.267034  ...   
2017-07-01   15.527558  0.251190  3808.096822  0.614131 -2696.027001  ...   
2017-08-01         NaN       NaN          NaN       NaN          NaN  ...   

greenbelt       York                                                    \
shape      E06000014                                                     
band            ndvi      ndwi      savi      ipvi        ci      gemi   
2017-04-01  0.419859 -0.434996  0.629708  0.328669  0.419859 -0.831839   
2017-05-01  0.754145 -0.699359  1.131079  0.671939  0.754145 -0.431212   
2017-06-01       NaN       NaN       NaN       NaN       NaN       NaN   
2017-07-01       NaN       NaN       NaN       NaN       NaN       NaN   
2017-08-01  0.621914 -0.604316  0.932758  0.527854  0.621914 -0.369069   

greenbelt                          Burton upon Trent and Swadlincote         
shape                                                      E07000193         
band             msavi      msavi2                             msavi msavi2  
2017-04-01         NaN         NaN                               NaN    NaN  
2017-05-01  751.027167  751.027167                               NaN    NaN  
2017-06-01         NaN         NaN                               NaN    NaN  
2017-07-01         NaN         NaN                               NaN    NaN  
2017-08-01         NaN         NaN                               NaN    NaN  

[5 rows x 5400 columns]

In [11]:
output_path = DIR_DATA / "processed_data/sentinel_two_indices_df_droppedna.hdf"
df_out.to_parquet(output_path)

In [12]:
df_in = pd.read_parquet(output_path)

In [13]:
np.testing.assert_array_equal(df_in.index.values, df_out.index.values)
np.testing.assert_array_equal(df_in.columns.values, df_out.columns.values)
np.testing.assert_array_equal(df_in.values, df_out.values)